# MinimumShareProduction

- Use `NemoMod MinimumShareProduction` to capture planned energy mix
- Note that there are some conflicting statements between sources:
    - 2023 IEA Energy transition plan says (p. 13): _Uganda would need to accelerate domestic end-use electrification efforts and greatly expand electricity exports to realise prior ambitions of installing 52 GW of generating capacity by 2040. These efforts could include earlier adoption of carbon capture, utilisation and storage (CCUS) and electrolytic hydrogen._
        - **NOTE** this is interpreted as _total capacity_ by 2040 as, on page 60, the report says: _The Energy Policy for Uganda 2023 states that by 2040 the country aims to have around 52 GW of installed capacity, which at healthy levels of utilisation would generate about 240 TWh annually_
- However, the Vision 2040 document states (list formatting added):
    - _The required capacity will be generated from different energy sources, namely_:
        - _hydro power (4500MW)_
        - _geo-thermal (1500 MW)_
        - _nuclear (24000 MW)_
        - _solar (5000 MW)_
        - _biomass (1700 MW)_
        - _peat (800 MW)_
        - _thermal (4300)_
    - Note that this totals 41.8 GW, ≈ 10.2 GW less than the ETP
- Additionally, installed capacity can guide production share, but it cannot define it

      

In [1]:
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import pathlib
import sisepuede.core.support_classes as sc
import sisepuede.manager.sisepuede_file_structure as sfs
import sisepuede.manager.sisepuede_models as sm
import sisepuede.utilities._toolbox as sf
import utils.common_data_needs as cdn
import warnings
warnings.filterwarnings("ignore")

from typing import *

plt.style.use("dark_background", )



/opt/miniconda3/envs/sisepuede/lib/python3.11/site-packages/sisepuede/core/model_attributes.py:6684: UserWarning: 

                        MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution 
                        implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() 
                        to ensure that transition probability rows are enforced on a simplex.
                        
                        
                        FIX THIS ASAP TO DERIVE PROPERLY.
                        
                        
  warnings.warn(
/opt/miniconda3/envs/sisepuede/lib/python3.11/site-packages/sisepuede/utilities/_toolbox.py:2551: UserWarning: Warning passed from optional_log: Successfully archived self.model_attributes to pickle at '/opt/miniconda3/envs/sisepuede/lib/python3.11/site-packages/sisepuede/out/sisepuede_run_2025-07-14T19;11;14.011803/sisepuede_run_2025-07-14T19;11;14.011803_model_attributes.pkl'.
  warnings.warn(f"Warning passed from optio

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling NemoMod...
Info Given NemoMod was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   1198.4 ms  ? NemoMod
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
/opt/miniconda3/envs/sisepuede/lib/python3.11/site-packages/sisepuede/utilities/_toolbox.py:2551: UserWarning: Warning passed from optional_log: Successfully initialized JuMP optimizer from solver module HiGHS..
  warnings.warn(f"Warning passed from optional_log: {msg}.")


In [2]:
dict_ssp = cdn._setup_sisepuede_elements()

matt = dict_ssp.get("model_attributes", )
models = dict_ssp.get("models", )
regions = dict_ssp.get("regions", )
time_periods = dict_ssp.get("time_periods", )

# setup region
_REGION_NAME = "uganda"
_REGION_ISO = regions.return_region_or_iso(_REGION_NAME, return_type = "iso")



# SET SOME GLOBALS

# fields

# year to build estimates out to
_YEAR_TARGET = 2100

# Load data we'll work with

- `

![screenshot_distribution_losses_20250709.png](./input_data/era/screenshot_distribution_losses_20250709.png)

In [ ]:
#


In [3]:
modvar_msp = matt.get_variable("NemoMod MinShareProduction")

In [165]:
df_transmission_loss = (
    pd.concat(
        [df_loss, df_proj],
        axis = 0,
    )
    .sort_values(by = [time_periods.field_year], )
    .reset_index(drop = True, )
)

df_transmission_loss.to_csv(
    cdn._PATH_OUTPUTS.joinpath(f"{modvar_tlf.name_fs_safe.upper()}.csv"),
    encoding = "UTF-8",
    index = None,
)